In [1]:
require 'nn'
require 'optim'
require 'mnist'
require 'dataset-mnist'

In [2]:
torch.manualSeed(0)
torch.setnumthreads(4)

In [3]:
classes = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20 }
--classes = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 }
geometry = { 32, 32 }

net = nn.Sequential()

net:add(nn.SpatialConvolution(1, 6, 5, 5))
net:add(nn.ReLU())
net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

net:add(nn.SpatialConvolution(6, 16, 5, 5))
net:add(nn.ReLU())
net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

net:add(nn.View(16*5*5))
net:add(nn.Linear(16*5*5, 120))
net:add(nn.ReLU())
net:add(nn.Linear(120, 84))
net:add(nn.ReLU())
net:add(nn.Linear(84, #classes))
net:add(nn.LogSoftMax())

In [4]:
parameters, gradParameters = net:getParameters()

In [5]:
criterion = nn.ClassNLLCriterion()

In [6]:
n_training_patches = 60000--60000
n_testing_patches = 10000--10000

In [12]:
pad = torch.zeros(10)

train_data = mnist.loadTrainSet(n_training_patches, geometry)
train_combined = torch.Tensor(n_training_patches * 2, 32, 32)
class_combined = torch.Tensor(n_training_patches * 2, 20)

for i = 1, n_training_patches do
    train_combined[i] = train_data[i][1]:clone()
    
    combined = torch.cat(train_data[i][2]:clone(), pad)
    class_combined[i] = combined
end

train_data = mnist.loadTrainSet(n_training_patches, geometry)

for i = 1, n_training_patches do
    k = 255 - train_data[i][1]
    
    combined = torch.cat(pad, train_data[i][2]:clone())
    train_combined[n_training_patches + i] = k
    class_combined[n_training_patches + i] = combined
end

mean = train_combined:mean()
stdev = train_combined:std()
print(mean, stdev)

train_combined:add(-mean)
train_combined:div(stdev)
print(#train_combined)

<mnist> done	


<mnist> done	


127.5	123.80376034192	


 120000
     32
     32
[torch.LongStorage of size 3]



In [13]:
test_data = mnist.loadTestSet(n_testing_patches, geometry)
test_combined = torch.Tensor(n_testing_patches * 2, 32, 32)
tclass_combined = torch.Tensor(n_testing_patches * 2, 20)

for i = 1, n_testing_patches do
    test_combined[i] = test_data[i][1]:clone()
    
    combined = torch.cat(test_data[i][2]:clone(), pad)
    tclass_combined[i] = combined
end

test_data = mnist.loadTestSet(n_testing_patches, geometry)

for i = 1, n_testing_patches do
    k = 255 - test_data[i][1]
    
    combined = torch.cat(pad, test_data[i][2]:clone())
    test_combined[n_testing_patches + i] = k
    tclass_combined[n_testing_patches + i] = combined
end

mean = test_combined:mean()
stdev = test_combined:std()
print(mean, stdev)

test_combined:add(-mean)
test_combined:div(stdev)
print(#test_combined)

<mnist> done	


<mnist> done	


127.5	123.82414832818	


 20000
    32
    32
[torch.LongStorage of size 3]



In [14]:
confusion = optim.ConfusionMatrix(classes)
epoch_limit = 5

In [15]:
actual_train = train_combined
actual_test = test_combined

In [16]:
batch_size = 10
for epoch = 1, epoch_limit do
    print("Epoch: " .. epoch)
    for t = 1, n_training_patches, batch_size do
        local inputs = torch.Tensor(batch_size * 2, 1, geometry[1], geometry[2])
        local targets = torch.Tensor(batch_size * 2)
        local k = 1

        for i = t, math.min(t + batch_size - 1, n_training_patches) do
            local input_normal = actual_train[i]:clone()
            local input_invert = actual_train[i + n_training_patches]:clone()
            
            local _, target_normal = class_combined[i]:clone():max(1)
            local _, target_invert = class_combined[i + n_training_patches]:clone():max(1)

            
            target_normal = target_normal:squeeze()
            target_invert = target_invert:squeeze()
            
            inputs[k] = input_normal
            inputs[k + 1] = input_invert
            
            targets[k] = target_normal
            targets[k + 1] = target_invert
            k = k + 2
        end

        local feval = function(x)
            collectgarbage()

            if x ~= parameters then
                parameters:copy(x)
            end
            gradParameters:zero()

            local outputs = net:forward(inputs)
            local f = criterion:forward(outputs, targets)
            local df_do = criterion:backward(outputs, targets)
            net:backward(inputs, df_do)

            for i = 1, batch_size * 2 do
                confusion:add(outputs[i], targets[i])
            end

            return f, gradParameters
        end

        sgd_state = sgd_state or {
            learning_rate = 0.06,
            momentum = 0,
            learning_rate_decay = 5e-7
        }
        optim.sgd(feval, parameters, sgd_state)
    end
    
    print(confusion)
    confusion:zero()
end


Epoch: 1	


ConfusionMatrix:
[[    1098     449      93      46      14      30     200    3904      85       4       0       0       0       0       0       0       0       0       0       0]   18.538% 	[class: 1]
 [    1034    1863      11      47       9       0       9    3717      48       4       0       0       0       0       0       0       0       0       0       0]   27.633% 	[class: 2]
 [     925     918     266     140      35       9     302    3077     264      22       0       0       0       0       0       0       0       0       0       0]   4.465% 	[class: 3]
 [     856     759      83     421      12      14      35    3686     211      54       0       0       0       0       0       0       0       0       0       0]   6.867% 	[class: 4]
 [     891     854       1      63     234       0     172    3414     115      98       0       0       0       0       0       0       0       0       0       0]   4.005% 	[class: 5]
 [     752     774      49     141      18      35      

  {
      1 : 1
      2 : 2
      3 : 3
      4 : 4
      5 : 5
      6 : 6
      7 : 7
      8 : 8
      9 : 9
      10 : 10
      11 : 11
      12 : 12
      13 : 13
      14 : 14
      15 : 15
      16 : 16
      17 : 17
      18 : 18
      19 : 19
      20 : 20
    }
  _prediction : FloatTensor - size: 20
  _pred_idx : LongTensor - size: 1
  nclasses : 20
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 20
  totalValid : 0.14545
}
Epoch: 2	


ConfusionMatrix:
[[    4976     118      84      43      14     197      99     226     155      11       0       0       0       0       0       0       0       0       0       0]   84.011% 	[class: 1]
 [       1    6132      88      56       9      53      15      38     327      23       0       0       0       0       0       0       0       0       0       0]   90.952% 	[class: 2]
 [     103     476    4179     166     156      57     631      68      96      26       0       0       0       0       0       0       0       0       0       0]   70.141% 	[class: 3]
 [      73     581     287    4050      29     262      29     329     298     192       1       0       0       0       0       0       0       0       0       0]   66.058% 	[class: 4]
 [      18     272      10       6    4315       2     382      76      52     708       1       0       0       0       0       0       0       0       0       0]   73.862% 	[class: 5]
 [     172     709     170     525     108    2540   

8
  classes : 
    {
      1 : 1
      2 : 2
      3 : 3
      4 : 4
      5 : 5
      6 : 6
      7 : 7
      8 : 8
      9 : 9
      10 : 10
      11 : 11
      12 : 12
      13 : 13
      14 : 14
      15 : 15
      16 : 16
      17 : 17
      18 : 18
      19 : 19
      20 : 20
    }
  _prediction : FloatTensor - size: 20
  _pred_idx : LongTensor - size: 1
  nclasses : 20
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 20
  totalValid : 0.70648333333333
}
Epoch: 3	


...

 : 
    {
      1 : 1
      2 : 2
      3 : 3
      4 : 4
      5 : 5
      6 : 6
      7 : 7
      8 : 8
      9 : 9
      10 : 10
      11 : 11
      12 : 12
      13 : 13
      14 : 14
      15 : 15
      16 : 16
      17 : 17
      18 : 18
      19 : 19
      20 : 20
    }
  _prediction : FloatTensor - size: 20
  _pred_idx : LongTensor - size: 1
  nclasses : 20
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 20
  totalValid : 0.87715
}
Epoch: 4	


ConfusionMatrix:
[[    5666       7      28      12       8      31      54      15      90      11       0       0       0       1       0       0       0       0       0       0]   95.661% 	[class: 1]
 [       2    6526      53      24       9      20       8      14      70      16       0       0       0       0       0       0       0       0       0       0]   96.796% 	[class: 2]
 [      48      48    5286      90     104      14     109      79     145      35       0       0       0       0       0       0       0       0       0       0]   88.721% 	[class: 3]
 [      19      44     129    5461       5     179      11      84     112      87       0       0       0       0       0       0       0       0       0       0]   89.072% 	[class: 4]
 [       6      25      45       1    5296       2      86      11      24     344       0       0       0       2       0       0       0       0       0       0]   90.654% 	[class: 5]
 [      52      40      39     148      43    4857   

ses : 
    {
      1 : 1
      2 : 2
      3 : 3
      4 : 4
      5 : 5
      6 : 6
      7 : 7
      8 : 8
      9 : 9
      10 : 10
      11 : 11
      12 : 12
      13 : 13
      14 : 14
      15 : 15
      16 : 16
      17 : 17
      18 : 18
      19 : 19
      20 : 20
    }
  _prediction : FloatTensor - size: 20
  _pred_idx : LongTensor - size: 1
  nclasses : 20
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 20
  totalValid : 0.908825
}
Epoch: 5	


ConfusionMatrix:
[[    5703       4      23       7       7      23      53      15      74      13       0       0       0       1       0       0       0       0       0       0]   96.286% 	[class: 1]
 [       3    6579      51      18      11       8       5      10      38      19       0       0       0       0       0       0       0       0       0       0]   97.582% 	[class: 2]
 [      38      40    5417      79      88       9      69      83     115      20       0       0       0       0       0       0       0       0       0       0]   90.920% 	[class: 3]
 [      16      29     110    5602       5     140       7      71      81      70       0       0       0       0       0       0       0       0       0       0]   91.372% 	[class: 4]
 [       6      17      45       0    5402       2      69      11      17     272       0       0       0       1       0       0       0       0       0       0]   92.468% 	[class: 5]
 [      34      29      26     109      29    5018   

ses : 
    {
      1 : 1
      2 : 2
      3 : 3
      4 : 4
      5 : 5
      6 : 6
      7 : 7
      8 : 8
      9 : 9
      10 : 10
      11 : 11
      12 : 12
      13 : 13
      14 : 14
      15 : 15
      16 : 16
      17 : 17
      18 : 18
      19 : 19
      20 : 20
    }
  _prediction : FloatTensor - size: 20
  _pred_idx : LongTensor - size: 1
  nclasses : 20
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 20
  totalValid : 0.928425
}


In [19]:
confusion:zero()

for t = 1, n_testing_patches, 10 do
    local inputs = torch.Tensor(batch_size * 2, 1, geometry[1], geometry[2])
    local targets = torch.Tensor(batch_size * 2)
    local k = 1
    
    for i = t, math.min(t + batch_size - 1, n_testing_patches) do
        --local sample = actual_test[i]
        local input_normal = actual_test[i]:clone()
        local input_invert = actual_test[i + n_testing_patches]:clone()
        
        
        local _, target_normal = tclass_combined[i]:clone():max(1)
        local _, target_invert = tclass_combined[i + n_testing_patches]:clone():max(1)
        
        target_normal = target_normal:squeeze()
        target_invert = target_invert:squeeze()
        
        inputs[k] = input_normal
        inputs[k + 1] = input_invert
        
        targets[k] = target_normal
        targets[k + 1] = target_invert
        k = k + 2
    end
    
    local preds = net:forward(inputs)
    
    for i = 1, batch_size * 2 do
        confusion:add(preds[i], targets[i])
    end
end

print(confusion)

ConfusionMatrix:
[[     967       1       1       0       0       1       2       1       7       0       0       0       0       0       0       0       0       0       0       0]   98.673% 	[class: 1]
 [       0    1126       3       2       0       0       1       0       2       1       0       0       0       0       0       0       0       0       0       0]   99.207% 	[class: 2]
 [      13       8     949      12      14       1       3       7      21       4       0       0       0       0       0       0       0       0       0       0]   91.957% 	[class: 3]
 [       3       7       8     962       1       3       0       7       6      13       0       0       0       0       0       0       0       0       0       0]   95.248% 	[class: 4]
 [       3       3       8       0     908       0       4       0       1      55       0       0       0       0       0       0       0       0       0       0]   92.464% 	[class: 5]
 [       8       5       1      35       3     812   

es : 
    {
      1 : 1
      2 : 2
      3 : 3
      4 : 4
      5 : 5
      6 : 6
      7 : 7
      8 : 8
      9 : 9
      10 : 10
      11 : 11
      12 : 12
      13 : 13
      14 : 14
      15 : 15
      16 : 16
      17 : 17
      18 : 18
      19 : 19
      20 : 20
    }
  _prediction : FloatTensor - size: 20
  _pred_idx : LongTensor - size: 1
  nclasses : 20
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 20
  totalValid : 0.92755
}
